In [1]:
import os
import numpy as np
from typing import Tuple
import tensorflow as tf
from sklearn.utils import class_weight

In [2]:
NUMBER_OF_TRAINING_DATA = 10
EPOCHS = 50

TRAINING_PATH = "../data_collection/datasets/training"
TEST_PATH = "../data_collection/datasets/test"

x_training = np.load(os.path.join(TRAINING_PATH, "class", "features.npy"), allow_pickle=True)
x_training = np.array([x_training[i].T for i in range(x_training.shape[0])])[-900:]
y_training = np.load(os.path.join(TRAINING_PATH, "class", "labels.npy"), allow_pickle=True)[-900:]

# Calculate class weights
class_weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y_training), y=y_training)

# Convert class weights to a dictionary
class_weights_dict = dict(enumerate(class_weights))

INPUT_DIM = x_training.shape[1:]

x_training = tf.convert_to_tensor(x_training.astype('float32'))
y_training = tf.convert_to_tensor(y_training.astype('int32'))



Metal device set to: Apple M2

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



In [3]:
class_weights = tf.cast(tf.constant(class_weights), "float32")
class_weights

<tf.Tensor: shape=(5,), dtype=float32, numpy=
array([1.6071428 , 1.5517242 , 0.46391752, 1.3533834 , 1.192053  ],
      dtype=float32)>

In [ ]:
def weightedLoss(weights=tf.constant([1,1,1,1,1])):
    def innerLoss(true, pred):
        w = tf.gather(weights, true) #(batch, labels, 1)

        loss = tf.keras.backend.sparse_categorical_crossentropy(true, pred) #(batch, labels)
        loss = w * loss

        return loss

    return innerLoss

In [ ]:


def create_conv1d_model(n:int, input_shape=Tuple[int, int]) -> tf.keras.Model:

    model = tf.keras.Sequential([
        tf.keras.layers.Conv1D(32, 2, 3, padding="same", input_shape=input_shape, activation="relu"),
        tf.keras.layers.MaxPooling1D(2, (3, 3)),
        tf.keras.layers.Conv1D(64, 2, 3, padding="same", input_shape=input_shape, activation="relu"),
        tf.keras.layers.MaxPooling1D(2, (3, 3)),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv1D(128, 2, 3, padding="same", input_shape=input_shape, activation="relu"),
        tf.keras.layers.MaxPooling1D(2, (3, 3)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(56, activation="relu"),
        tf.keras.layers.Dense(1, activation="sigmoid")
    ])

    model.compile(
        loss="mse",
        optimizer=tf.keras.optimizers.Adam(),
        metrics=["acc", "val_acc"]
    )

    return model